In [2]:
import numpy as np
import pandas as pd

In [3]:

from typing import List, Dict

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
songs = pd.read_csv('/content/songdata.csv')

In [6]:

songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
songs['text'] = songs['text'].str.replace(r'\n', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [9]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [10]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [11]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [12]:
similarities = {}

In [13]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [14]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)


In [15]:
recommedations = ContentBasedRecommender(similarities)

In [21]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

songs['song'].iloc[10]

'Saints In Hell'

In [17]:
recommedations.recommend(recommendation)

The 4 recommended songs for Saints In Hell are:
Number 1:
I Don't Know Why by Erasure with 0.222 similarity score
--------------------
Number 2:
Hell's Ditch by Pogues with 0.177 similarity score
--------------------
Number 3:
I'd Cry If You Died by Devo with 0.163 similarity score
--------------------
Number 4:
Hell Was Made In Heaven by Helloween with 0.155 similarity score
--------------------


In [22]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}
songs['song'].iloc[120]

'My Hitta'

In [19]:
recommedations.recommend(recommendation2)

The 4 recommended songs for My Hitta are:
Number 1:
Act Right by Yo Gotti with 0.044 similarity score
--------------------
Number 2:
Leave The Club by Chris Brown with 0.036 similarity score
--------------------
Number 3:
13 Years Of Grief by Zakk Wylde with 0.032 similarity score
--------------------
Number 4:
Lay You Down by Usher with 0.03 similarity score
--------------------
